In [1]:
"""This file is a demo for using the DG whole body controller.

License BSD-3-Clause
Copyright (c) 2021, New York University and Max Planck Gesellschaft.

Author: Julian Viereck
Date:   Feb 24, 2021
"""

import numpy as np
np.set_printoptions(suppress=True, precision=3)

import pinocchio as pin

import dynamic_graph as dg

from robot_properties_solo.config import Solo12Config
from dg_blmc_robots.solo.solo12_bullet import get_solo12_robot

from reactive_planners.dynamic_graph.solo12_stepper import Solo12WBCStepper

from dg_tools.dynamic_graph.dg_tools_entities import CreateWorldFrame

In [3]:
###
# Create the simulated robot
robot = get_solo12_robot()

In [4]:
q0 = Solo12Config.q0.copy()
q0[0] = 0.2

# Rotate the base if desired.
des_yaw = 0.

q0[3:7] = pin.Quaternion(pin.rpy.rpyToMatrix(0., 0., des_yaw)).coeffs() # 

robot.reset_state(q0, Solo12Config.v0)

In [5]:
ctrl = Solo12WBCStepper('solo12_wbc_stepper', 0.2)

In [6]:
# Zero the initial position from the vicon signal.
base_posture_sin, base_velocity_sin = robot.base_signals()

op = CreateWorldFrame('wf')
dg.plug(base_posture_sin, op.frame_sin)
op.update()
op.set_which_dofs(np.array([1., 1., 0., 0., 0., 0.]))

base_posture_local_sin = stack_two_vectors(
    selec_vector(subtract_vec_vec(base_posture_sin, op.world_frame_sout), 0, 3),
    selec_vector(base_posture_sin, 3, 7), 3, 4)

In [7]:
ctrl.plug(robot, base_posture_local_sin, base_velocity_sin)

In [8]:
# Set desired base rotation.
ctrl.des_ori_pos_rpy_sin.value = np.array([0., 0., des_yaw])
ctrl.des_com_vel_sin.value = np.array([0.0, 0.0, 0.])

In [9]:
# Simulate for 1 seconds.
robot.run(1000, sleep=True)

In [10]:
# Start the stepper.
ctrl.start()

In [12]:
# Simulate for 4000 seconds.
robot.run(4000, sleep=True)